In [1]:
import pandas as pd
import numpy as np
import pickle5 as pickle
from sklearn.model_selection import train_test_split

In [2]:
'''
    Optional: Steps to prepare data.
    Alternatively, read in already processed file 3 cells below
'''

import preprocessor as prep

raw_data = prep.read_data()
raw_data = prep.clear_columns(raw_data)

# Apply bandpass filter: limit 250Hz
raw_data = prep.apply_bp_filter(raw_data)

# Clean edges
raw_data = prep.clear_edges(raw_data)

# Normalize raw data
raw_data = prep.normalize_data(raw_data)

# ??
picture_blocks = prep.extract_picture_blocks(raw_data)
data_samples = prep.extract_data_samples(picture_blocks, sample_length=2, sample_step=2)

# Extract 15 features in features.py
final_data = prep.extract_features_for_all_data(data_samples)
final_data.head()

################################################################
Step 1 - Collecting data from .txt files
Step 2 - Done
################################################################

Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extracting features...
Extractin

,apen,fmean,fmed,fmode,intrange,mav,mavfd,mavsd,peak,rms,sampen,srange,std,var,zc,Emotion,Muscle,Person,Sample idx
0,0.348922,0.297715,0.300781,97.645657+65.859915j,0.733160,0.331975,0.539612,0.874312,0.565392,0.369964,0.375472,1.157271,0.369963,0.136873,299,happy,Zygomaticus Major,aun,0
1,0.226602,0.300070,0.300781,67.496357+95.333692j,0.698057,0.327800,0.533154,0.860990,0.566129,0.366780,0.245257,1.134112,0.366780,0.134527,300,happy,Zygomaticus Major,aun,1
2,0.324885,0.299863,0.300781,30.513549+23.107629j,0.753091,0.329021,0.535712,0.868040,0.586717,0.366916,0.336129,1.198740,0.366915,0.134627,299,happy,Zygomaticus Major,aun,2
3,0.292069,0.299166,0.300781,24.674829+26.778399j,0.713777,0.330327,0.536233,0.867013,0.618406,0.368607,0.314431,1.205682,0.368602,0.135868,300,happy,Zygomaticus Major,aun,3
4,0.539241,0.296760,0.300781,72.088227+80.914239j,0.662158,0.303459,0.491866,0.796976,0.553420,0.339462,0.603989,1.155964,0.339447,0.115225,300,happy,Zygomaticus Major,aun,4


In [3]:
'''
    Optional [Step 2]: Combine zygomaticus & corrugator readings for selected windows
'''

from sklearn.preprocessing import LabelEncoder

participants = set(final_data.Person)
emotions = set(final_data.Emotion)
muscles = list(set(final_data.Muscle))

formatted_df = None
s = None

for participant in participants:
    for emotion in emotions:
        s1 = final_data[(final_data.Person == participant) & (final_data.Emotion == emotion)\
                        & (final_data.Muscle == muscles[0])]
        s2 = final_data[(final_data.Person == participant) & (final_data.Emotion == emotion)\
                        & (final_data.Muscle == muscles[1])]
        s = pd.merge(s1, s2, on=['Sample idx', 'Emotion', 'Person'])
        s = s.drop(columns=['Muscle_x', 'Muscle_y', 'Sample idx', 'Person'])
    
        if formatted_df is None:
            formatted_df = s
        else:
            formatted_df = pd.concat([formatted_df, s])

emotions_le = LabelEncoder()
formatted_df.Emotion = emotions_le.fit_transform(formatted_df.Emotion)
formatted_df.head()

,apen_x,fmean_x,fmed_x,fmode_x,intrange_x,mav_x,mavfd_x,mavsd_x,peak_x,rms_x,...,mav_y,mavfd_y,mavsd_y,peak_y,rms_y,sampen_y,srange_y,std_y,var_y,zc_y
0,1.319408,0.227780,0.238281,6.585242-0.128821j,0.159700,0.104038,0.138425,0.221761,0.624339,0.139013,...,0.144971,0.103643,0.164611,0.509589,0.175628,1.768503,1.069221,0.175499,0.030800,109
1,1.305951,0.192242,0.179688,3.147888-2.052306j,0.071008,0.052543,0.063242,0.097658,0.516814,0.078765,...,0.080253,0.070221,0.112050,0.366768,0.103402,1.762655,0.795979,0.103141,0.010638,145
2,1.278537,0.199957,0.226562,6.077910+1.538279j,0.105875,0.074470,0.099952,0.160935,0.378509,0.102758,...,0.088945,0.083415,0.134376,0.402705,0.115325,1.904622,0.899944,0.115108,0.013250,145
3,1.310553,0.235445,0.253906,6.289809+0.360504j,0.138544,0.088891,0.124476,0.196624,0.400775,0.117034,...,0.100175,0.096489,0.153011,0.443350,0.127829,1.964340,0.789363,0.127776,0.016327,156
4,1.182922,0.238121,0.234375,5.165706+2.315500j,0.120339,0.088080,0.125521,0.201664,0.646654,0.124639,...,0.076642,0.093466,0.151310,0.483104,0.100315,2.030011,0.845174,0.100271,0.010054,210


In [ ]:
'''
    Optional: Best Model from TPOT
    Ensemble Classifier from TPOT
    Current Mean Accuracy: 0.79
'''

import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = pd.read_csv('preprocessed_data.csv')
features = tpot_data.drop('Emotion', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['Emotion'], random_state=1)

# Average CV score on the training set was: 0.6226190476190475
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=RandomForestClassifier(bootstrap=False, criterion="entropy", max_features=0.7500000000000001, min_samples_leaf=9, min_samples_split=13, n_estimators=100)),
    GradientBoostingClassifier(learning_rate=0.1, max_depth=4, max_features=0.5, min_samples_leaf=2, min_samples_split=16, n_estimators=100, subsample=1.0)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 1)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

# Mean Accuracy
print('Mean accuracy: %r' % np.mean(results == testing_target.values))

In [5]:
'''
    Read processed file
'''

try:
    df = pd.read_pickle("data_sample_len2_sample_step2.pkl")
except:
    try:
        with open('data_sample_len2_sample_step2.pkl', 'rb') as fh:
            df = pickle.load(fh)
    except:
        print('Cannot open pickle file')

In [51]:
df["Muscle"] = df["Muscle"].astype('category')
df["Emotion"] = df["Emotion"].astype('category')
df = df.drop(["Person", "Sample idx"], axis=1)

X = pd.get_dummies(df.drop(["Emotion", "fmode"], axis=1))
y = df["Emotion"]

In [9]:
df.head(n=3)

,apen,fmean,fmed,fmode,intrange,mav,mavfd,mavsd,peak,rms,sampen,srange,std,var,zc,Emotion,Muscle
0,1.307227,0.238228,0.253906,11.391353+7.735965j,0.296268,0.184651,0.253215,0.396857,1.000022,0.233531,1.939744,1.718309,0.233519,0.054531,238,happy,Zygomaticus Major
1,1.326351,0.244564,0.261719,14.179856+12.743834j,0.356517,0.205449,0.280853,0.439035,0.704597,0.257154,2.101206,1.537603,0.257139,0.066121,242,happy,Zygomaticus Major
2,1.252011,0.243792,0.269531,14.682492+0.998472j,0.342912,0.205583,0.298171,0.466317,0.912664,0.262910,1.887932,1.691022,0.262871,0.069101,258,happy,Zygomaticus Major
3,1.325152,0.076673,0.015625,10.933231-0.048197j,0.206222,0.136505,0.147402,0.237890,0.564521,0.181422,1.931197,1.221809,0.181422,0.032914,187,happy,Zygomaticus Major
4,1.279311,0.227714,0.246094,12.965370+10.648279j,0.210121,0.129111,0.186362,0.294306,0.442515,0.166861,1.988861,1.005221,0.166834,0.027834,258,happy,Zygomaticus Major
5,1.279078,0.243330,0.257812,20.661735-0.038338j,0.208477,0.127903,0.170512,0.271788,0.582802,0.163754,1.914128,1.027060,0.163351,0.026683,246,happy,Zygomaticus Major
6,1.289047,0.149394,0.152344,9.940519+0.146295j,0.322100,0.187468,0.214039,0.338752,0.862564,0.233311,1.924678,1.559338,0.233082,0.054327,202,happy,Zygomaticus Major
7,1.206356,0.117703,0.046875,10.952140+6.187237j,0.197548,0.108668,0.064651,0.103962,0.288620,0.130342,1.404475,0.637425,0.130329,0.016986,64,neutral,Zygomaticus Major
8,1.248492,0.087167,0.007812,13.232429+2.899074j,0.089472,0.054208,0.059050,0.094815,0.220781,0.068833,1.654984,0.358381,0.067974,0.004620,204,neutral,Zygomaticus Major
9,1.248380,0.199107,0.300781,7.806749+10.534022j,0.083308,0.051128,0.056432,0.090647,0.197932,0.066204,1.690715,0.457629,0.066022,0.004359,223,neutral,Zygomaticus Major


In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [96]:
from sklearn.svm import SVC

clf = SVC(kernel="rbf")
clf.fit(X_train, y_train)
print(np.mean(clf.predict(X_test) == y_test))

0.2972972972972973


In [97]:
clf = SVC(kernel="linear")
clf.fit(X_train, y_train)
print(np.mean(clf.predict(X_test) == y_test))

0.32432432432432434


In [98]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(X_train, y_train)
print(np.mean(clf.predict(X_test) == y_test))

0.4594594594594595


In [4]:
from sklearn.ensemble import RandomForestClassifier

X = formatted_df.drop(["Emotion", "fmode_x", "fmode_y"], axis=1)
y = formatted_df["Emotion"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

clf = RandomForestClassifier(n_estimators=50, random_state=5)
clf = clf.fit(X_train, y_train)
np.mean(clf.predict(X_test) == y_test)

0.7380952380952381

In [10]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=len(y), random_state=0, shuffle=True)
accuracies = []

for train_index, test_index in kf.split(X, y):
    clf = RandomForestClassifier(n_estimators=100, random_state=0)
    X_train, X_test = X.values[train_index], X.values[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    acc = np.mean(clf.predict(X_test) == y_test)
    accuracies.append(acc)

In [11]:
np.mean(accuracies)

0.5277777777777778

In [12]:
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold

rf = RandomForestClassifier(n_estimators=100)
seed = 0
param_grid = {"min_samples_split":[2,4,8,16,32] }

# Inner CV: for hyperparameter tuning inside the leave-one-out CV
inner_cv = KFold(n_splits=10, shuffle=True, random_state=seed)

# Outer CV: leave-one-out CV
outer_cv = KFold(n_splits=len(y), shuffle=True, random_state=seed)

# Non-nested parameter search: just leave-one-out CV
clf = GridSearchCV(estimator=rf, param_grid=param_grid, cv=inner_cv)
clf.fit(X, y)
non_nested_score = clf.best_score_

# Nested CV
clf = GridSearchCV(estimator=rf, param_grid=param_grid, cv=inner_cv)
nested_score = cross_val_score(clf, X=X, y=y, cv=outer_cv).mean()

In [ ]:
#!pip insall tpot

In [16]:
'''
    Optional: Build Model pipeline with tpot
'''

from tpot import TPOTClassifier

X = formatted_df.drop(["Emotion", "fmode_x", "fmode_y"], axis=1)
y = formatted_df["Emotion"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

pipeline_optimizer = TPOTClassifier(generations=40, population_size=20, cv=20,
                                    random_state=1, verbosity=2, scoring='accuracy')
pipeline_optimizer.fit(X_train, y_train)

# Mean Accuracy
results = pipeline_optimizer.predict(X_test)
print('Mean accuracy: %r' % np.mean(results == y_test))

# Save pipeline
# pipeline_optimizer.export("pipeline.py")

[19:47:51] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:47:52] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:47:52] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:47:52] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the object

[19:47:52] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:47:52] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:47:52] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:47:52] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the object

[19:48:08] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:48:08] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:48:08] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:48:08] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the object

[19:48:12] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:48:12] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:48:12] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:48:12] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the object

[19:48:58] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:48:58] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:48:58] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:48:58] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the object

...

[20:07:15] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:07:15] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:07:15] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:07:15] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the object

[20:17:50] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:17:50] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:17:50] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:17:50] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the object

[20:22:23] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:22:24] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:22:24] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:22:25] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the object

[20:37:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:37:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:37:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:37:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the object

Generation 32 - Current best internal CV score: 0.6226190476190475
Generation 33 - Current best internal CV score: 0.6226190476190475
Generation 34 - Current best internal CV score: 0.6226190476190475
Generation 35 - Current best internal CV score: 0.6226190476190475
Generation 36 - Current best internal CV score: 0.6226190476190475
[21:12:27] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:14:54] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:14:54] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0